### Remove tweets with more than one drug

The model was trained on HCQ data, and we used the unmasked model for inference. So we need to remove tweets with multiple drug names.

In [3]:
import glob
import pandas as pd

# dfs = []
# for file in glob.glob("../data/final/*.csv"):
#     df = pd.read_csv(file)[["wave","stance"]].value_counts().reset_index().sort_values(["wave","stance"]).reset_index(drop=True)
#     df["drug"] = file.split("/")[-1].split(".csv")[0]
#     dfs.append(df)

In [1]:
df = pd.concat(dfs)
df = df.sort_values(["drug","wave","stance"])
dfs = []
for n,g in df.groupby(["drug","wave"]):
    s = g[0].sum()
    g["ratio"] = g[0]/s
    dfs.append(g.reset_index())
df = pd.concat(dfs)
df


In [18]:
df.to_csv("distribution.csv", index=False)

In [6]:
df.drug = df.drug.replace("hcq","Hydroxychloroquine")
df.drug = df.drug.replace("ivermectin","Ivermectin")
df.drug = df.drug.replace("molnupiravir","Molnupiravir")
df.drug = df.drug.replace("remdesivir","Remdesivir")


In [14]:
df["Count"] = df.pop("level_0")

In [15]:
df

,wave,stance,drug,ratio,Count
0,1,-1,Hydroxychloroquine,0.350514,0
1,1,0,Hydroxychloroquine,0.138435,1
2,1,1,Hydroxychloroquine,0.511050,2
3,2,-1,Hydroxychloroquine,0.328862,0
4,2,0,Hydroxychloroquine,0.133797,1
5,2,1,Hydroxychloroquine,0.537341,2
6,3,-1,Hydroxychloroquine,0.320499,0
7,3,0,Hydroxychloroquine,0.136435,1
8,3,1,Hydroxychloroquine,0.543065,2
9,1,-1,Ivermectin,0.123950,0


In [10]:
# df = pd.concat(dfs)
# dfs = []
# for n,g in df.groupby(["Drug","Wave"]):
#     g = g.sort_values("Stance",ascending=True)
#     dfs.append(g.reset_index())
# df = pd.concat(dfs)
# d={-1:"Negative",
# 0:"Neutral",
# 1:"Positive"}
# def map_stance(x):
#     return d[x]
# df.Stance=df.Stance.apply(lambda x: map_stance(x))

In [11]:
df["Number of tweets"] = df.pop("Count")

In [20]:
# df.pop("index")
df.columns = [c.capitalize() for c in df.columns]

In [63]:
import altair as alt


plot1 = alt.Chart(df).mark_bar().encode(
    x = "Wave:O",
    y = "Ratio:Q",
    color=alt.Color('Stance:N',scale = alt.Scale(domain=[-1,0,1],range=["#fd7f6f","#cfd8dc","#7e9ab9"])),
    opacity=alt.value(0.9),
    column = "Drug:N"
).properties(
    width=60,
    height=150
)

plot1

alt.Chart(...)

In [225]:
import pandas as pd

DRUG = "hcq" ## specify the dataset name.
data_dir = "../data/with_location/"

In [18]:
keywords_dict = {"hcq":"ydroxych| hcq |plaqu |plaquenil|hydroquin|axemal",
                    "ivermectin": "ivermectin|stromectol|soolantra|sklice",
                    "remdesivir": "remdesivir|veklury",
                    "molnupiravir": "molnupiravir|merck's drug|merck's pill|merck's antiviral"}
for drug, keywords in keywords_dict.items():
    df = pd.read_csv(f"{data_dir}{drug}.csv")
    # drug_df = df[df["full_text"].str.contains(keywords, case=False)].drop_duplicates()
    other_keys = keywords_dict.copy()
    other_keys.pop(drug)
    keys_for_other_drugs = "|".join(list(other_keys.values()))
    df = df[~df.full_text.str.contains(keys_for_other_drugs, case=False)]
    if not df.empty:
        df.to_csv(f"{data_dir}{drug}.csv", index=False)
        message = "%i tweets written to %s\n"%(len(df), f"{data_dir}{drug}.csv")
        print(message)
    else:
        print("Nothing left. Skipping...")
    break

/mnt/enchome/anaconda3/envs/twee/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


321040 tweets written to ../data/with_location/hcq.csv



In [22]:
df = df.dropna(subset=["state"])

In [4]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize()
import re

def clean_url(x):
    return re.sub(r"http\S+","", x)

df = pd.read_csv(f"{data_dir}{DRUG}.csv")

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# model_masked = AutoModelForSequenceClassification.from_pretrained("model/Twitter-drug-stance-bert-masked").to("cuda:1")  ## or masked 
# model_masked.eval()
model = AutoModelForSequenceClassification.from_pretrained("model/Twitter-drug-stance-bert").to("cuda:1")  ## or masked 
model.eval()

/mnt/enchome/anaconda3/envs/twee/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [24]:
def infer(x):
    tok = tokenizer(x,return_tensors = 'pt')
    tok.to("cuda:1")
    
    output = model(**tok)
    predictions = torch.argmax(output.logits, dim=-1)
    
    return int(predictions[0])

# def masked_infer(x):
#     tok = tokenizer(x,return_tensors = 'pt')
#     tok.to("cuda:1")
    
#     output = model_masked(**tok)
#     predictions = torch.argmax(output.logits, dim=-1)
    
#     return int(predictions[0])

In [29]:
from tqdm.auto import tqdm
tqdm.pandas()
df["stance"] = df.full_text.progress_apply(lambda x:infer(x))

100%|██████████| 292289/292289 [1:07:56<00:00, 71.70it/s] 
<ipython-input-29-9af531ff6ba8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["stance"] = df.full_text.progress_apply(lambda x:infer(x))


In [31]:

from pandarallel import pandarallel
pandarallel.initialize()


def re_map(x):
    if x==1: return -1
    elif x==2: return 1
    else: return 0
df.stance = df.stance.parallel_apply(lambda x: re_map(x))
df.stance.value_counts()  



INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/mnt/enchome/anaconda3/envs/twee/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


 1    151878
-1    100320
 0     40091
Name: stance, dtype: int64

In [33]:
df.to_csv(f"../data/final/{DRUG}.csv",index=False)

## Evaluation


In [18]:
import glob
import pandas as pd
from tqdm import tqdm
import os

tqdm.pandas()
evaluation_dir = "evaluation/"
os.makedirs(evaluation_dir, exist_ok=True)

for drug in ["hcq","ivermectin","molnupiravir","remdesivir"]:
    df = pd.read_csv(f"{data_dir}{drug}.csv").sample(100, random_state=42)
    df["stance"] = df.full_text.apply(lambda x: infer(x))   
    df["masked_prediction"] = df.full_text.progress_apply(lambda x: masked_infer(x))
    df["prediction"] = df.full_text.progress_apply(lambda x: infer(x))
    df.to_csv(f"{evaluation_dir}{drug}.csv",index=False)

100%|██████████| 100/100 [00:01<00:00, 93.95it/s]
